###### Copyright &copy; Anand B Pillai, Anvetsu Technologies Pvt. Ltd (2015)

# Classes & Types

# 1. Inaccessible Method

### 1.1 Show me the Code !

In [5]:
class C(object):
        
    def add(self, x, y):
        # All good
        return x + y
    
    def sub(x, y):
        # Forgot the explicit 'self' param
        return x - y

In [6]:
c=C()
c.add(50, 60)

110

In [7]:
c.sub(50, 60) # Won't work

TypeError: sub() takes exactly 2 arguments (3 given)

In [8]:
# O.K - let me try passing an explicit self to it.
c.sub(c, 50, 60)   # Nope

TypeError: sub() takes exactly 2 arguments (4 given)

In [9]:
# O.K - Maybe calling via class works ?
C.sub(50, 60)

TypeError: unbound method sub() must be called with C instance as first argument (got int instance instead)

In [10]:
# O.K - Maybe this works ?
C.sub(c, 50, 60)

TypeError: sub() takes exactly 2 arguments (3 given)

## Gotcha !!!

#### This is because,

   1. When a method is defined in a class, Python __always__ passes the implicit self argument to it as the first parameter.
   1. If you want a function to sit inside a class and NOT take the implicit first argument, redefine it as a __static method__ .
   1. Otherwise for all instance methods, don't forget to add the implicit __self__ argument explicitly while writing code :)

### 1.1 Show me the Fix !

In [11]:
# 1. Make the method properly bound to the instance by explicit self.
class C(object):
        
    def add(self, x, y):
        return x + y
    
    def sub(self, x, y):
        return x - y

In [12]:
c=C()
c.sub(60, 50)

10

In [17]:
# 1. Make the method an unbound method by making it static
class C(object):
        
    def add(self, x, y):
        return x + y
    
    @staticmethod
    def sub(x, y):
        # Dont add the self param for a static method.
        return x - y

In [18]:
c=C()
c.sub(60, 50)

10

In [19]:
# A static method is a like a free floating function bound to a class namespace. 
# You can call it from the class as well.
C.sub(100, 20)

80

# 2. Nothing is Really Private

#### When you start learning Python classes, one of the first questions asked is,

   1. How to create a variable private inside a class ?
   
#### The common answer is to say, prefx a double underscore to it. So if your variable is "__x__" making it "______x" hides it. Unfortunately Python never actually hides it.

### 2.1 Show me the Code !

In [88]:
class Name(object):
    
    def __init__(self, first, second):
        # Private variables
        self.__first = first
        self.__second = second
        
    def __str__(self):
        return "%s %s" % (self.__first, self.__second)
    
    def get_first(self):
        # Accessor to first name
        return self.__first
    
    def get_second(self):
        # Accessor to second name
        return self.__second
    

In [92]:
n = Name('Narendra','Modi')
print n

Narendra Modi


In [93]:
print n.get_first()
print n.get_second()

Narendra
Modi


In [94]:
# Try to access __first
n.__first

AttributeError: 'Name' object has no attribute '__first'

In [96]:
# However,
n._Name__first  # How ?

'Narendra'

## Gotcha !!!

#### This is because,

   1. Python classes have no true name hiding of private attributes.
   1. Instead Python only performs a __name mangling__ if it finds an attribute prefixed with a double underscore. 
   1. Typically the mangling is done as,
    
    _____classname______var, where __var__ is the variable name without the underscore.
         

In [97]:
# Hence in this case __first is available as _Name__first
# and __second as _Name__second.
print n._Name__first
print n._Name__second

Narendra
Modi


# 3. Class - Variable Definiton & Binding

### 3.1 Show me the Code !

In [118]:
import uuid

class Employee(object):
    """ A class describing an Employee """
    
    def __init__(self, name):
        self.name = name
        
    def get_id(self):
        e_id = EmployeeID()
        return e_id.get_id()
    
class EmployeeID(object):
    """ A class which makes a unique Employee system ID """
    
    def get_id(self):
        return uuid.uuid4().hex


In [119]:
# Good
e = Employee('Anand')
e.get_id()

'bd40491d9ad146858b56c1dac6007e6a'

In [122]:
import uuid

class Employee(object):
    """ A class describing an Employee """
    
    # Let us make use of a class variable to do this now
    e_id = EmployeeID2()
    
    def __init__(self, name):
        self.name = name
        
    def get_id(self):
        return self.e_id.get_id()
    
class EmployeeID2(object):
    """ A class which makes a unique Employee system ID """
    
    def get_id(self):
        return uuid.uuid1().hex


NameError: name 'EmployeeID2' is not defined

## Gotcha !!!

#### This is because,

   1. In the first case, the __EmployeeID__ instance is created inside an instance method. Instance methods are evaluated only after Python has parsed the entire code so by the time the method is intered the definition of __EmployeeID__ class which follows the __Employee__ class is already available.
   1. In the second case, the __EmployeeID2__ instance is a class-level variable. In Python, the body of a class, including any class variables is evaluated the moment it is encountered. Python tries to create an instance of __EmployeeID2__ class immediately upon parsing the class __Employee__. Since the definition comes later and is not yet available, Python throws a __NameError__ .
         

### 3.1 Show me the Fix !

In [123]:
# The fix is simple - define the dependee class before the dependent class in order.
# In this case, define EmployeeID2 before Employee .

import uuid

class EmployeeID2(object):
    """ A class which makes a unique Employee system ID """
    
    def get_id(self):
        return uuid.uuid1().hex

class Employee(object):
    """ A class describing an Employee """
    
    # Let us make use of a class variable to do this now
    e_id = EmployeeID2()
    
    def __init__(self, name):
        self.name = name
        
    def get_id(self):
        return self.e_id.get_id()

In [124]:
e = Employee('Anand')
e.get_id()

'9b04fc46bb1c11e497b8c4850885d694'

In [125]:
# Note - A similar problem occurs for ANY undefined dependency - it can be
# a function as well - here is another example.

import uuid

class Employee(object):
    """ A class describing an Employee """
    
    # Let us make use of a class variable to do this now
    e_id = get_id()
    
    def __init__(self, name):
        self.name = name
        
    def get_id(self):
        return self.e_id
     
def get_id(self):
    """ A function making a unique ID """
    
    return uuid.uuid1().hex

NameError: name 'get_id' is not defined

#### More Reading

   1. https://mail.python.org/pipermail/tutor/2009-August/071266.html

# 4. Types and classes (warning: Advanced)

### 4.1 Show me the Code !

In [98]:
class C(object): 
    pass

In [99]:
# Make an instance
c = C()
# Inspect the __class__ of C
c.__class__ 

__main__.C

In [100]:
# OK
isinstance(c, C)

True

In [101]:
# Now check the class of class C itself !
C.__class__

type

In [102]:
issubclass(C, type) 

False

In [103]:
# Hmmm - but C.__class__ returned type ?
isinstance(C, type)

True

## Gotcha !!!

#### This is because,

   1. Python makes a distinction between __types__ and __classes__ . 
   1. Think of types as class factories - i.e classes are instances of types (brain-melting point !!!)
   1. C is derived from __object__, which is an instance of the most basic type __type__ .
   1. Hence the ______class____ of __C__ is __type__, as in a class-instance relationship. But C is derived from the __object__ class (most basic class in Python).
   1. __object__ itself is a class which is an instance, (but no a sub-class) of __type__ :) 

In [107]:
issubclass(C, object)

True

In [105]:
# So is object an instance of type ?
# Yes
isinstance(object, type)

True

In [106]:
# So, is object a subclass of type ?
# No
issubclass(object, type)

False

In [108]:
# Type is an instance of itself
isinstance(type, type)

True

In [109]:
# As well a sub-class of itself :)
issubclass(type, type)

True

###### Copyright &copy; Anand B Pillai, Anvetsu Technologies Pvt. Ltd (2015)